In [1]:
import tensorflow as tf

In [2]:
device_list=tf.test.gpu_device_name()


In [3]:
device_list

'/device:GPU:0'

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import re
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix


In [5]:
import os
print(os.listdir('/kaggle/input'))

['test-data2', 'train-data3']


In [6]:
import pandas as pd
data=pd.read_csv('/kaggle/input/train-data3/train_dataset3.csv')



In [7]:
data.head()

id               user_name                 user_location  \
0  1365695382076854280      Seeji Sundarakshan              Mumbai & Kerala.   
1  1379853509902934025             Tim Niblett  Burlington ON and Hudson FL    
2  1446041875815743491  KAS Sub-Saharan Africa               Berlin, Germany   
3  1408683429072232448          CowinBangalore                           NaN   
4  1410637131131805705                  🐝Lisa🐝                        Jersey   

                                    user_description         user_created  \
0  interest in geopolitics, economics, independen...  2016-09-04 09:39:11   
1                                                NaN  2011-03-28 20:43:51   
2  Official account Sub-Saharan Africa Dept. @KAS...  2021-05-31 14:19:29   
3  Follow us to get update as soon as 18 plus vac...  2021-05-17 18:56:45   
4          A happy Bean living in a beautiful island  2009-05-02 15:15:26   

   user_followers  user_friends  user_favourites  user_verified  \
0             608           277            27106          False   
1              88           219             5375          False   
2             312           195              412          False   
3            1102             2               57          False   
4              41           489             1940          False   

                  date  ...               source retweets favorites  \
0  2021-02-27 16:08:47  ...      Twitter Web App        0         1   
1  2021-04-07 17:48:07  ...  Twitter for Android        0         2   
2  2021-10-07 09:17:04  ...      Twitter Web App        0         3   
3  2021-06-26 07:07:56  ...    cowin_vaccine_app        0         0   
4  2021-07-01 16:31:15  ...   Twitter for iPhone        0         0   

   is_retweet                              cleaned_text_textblob  polarity  \
0       False  consolation good number sputnikv instead going...      0.70   
1       False  wife got 2nd moderna shot yesterday florida . ...      0.25   
2       False  following author study tom bayes , china prese...      0.00   
3       False  560011 , age 45 41d1 , 276d2 slot , covaxin 30...      0.00   
4       False                happy 2nd vaccine day ! ! ! moderna      0.40   

  subjectivity  Sentiment_textBlob  \
0         0.60            Positive   
1         0.25            Positive   
2         0.05             Neutral   
3         0.00             Neutral   
4         0.50            Positive   

                                  cleaned_text_vader Sentiment_vader  
0  the only consolation is that good number of sp...        Positive  
1  my wife and i both got our 2nd moderna shots y...        Positive  
2  following the author of the study tom bayes, c...        Negative  
3  560011, age 45 41d1, 276d2 slots, covaxin on 3...         Neutral  
4             happy 2nd vaccine day to me!!! moderna        Positive  

[5 rows x 22 columns]

In [8]:
# Conserver uniquement les colonnes utiles (par exemple, 'text' et 'sentiment')
data = data[['text', 'Sentiment_vader']]

In [9]:
data.head(20)

text Sentiment_vader
0   @IsabelO20183051 The only consolation is that ...        Positive
1   My wife and I both got our 2nd #Moderna shots ...        Positive
2   Following the author of the study Tom Bayes, #...        Negative
3   560011, \nAGE 45+\n41-D1, 276-D2 slots,\n#COVA...         Neutral
4             Happy 2nd Vaccine Day to me!!! #Moderna        Positive
5   @sepehrifar @hrw @MichaelARPage @ksadjadpour @...        Negative
6   Excellent work by Ayudh Volunteers! ...\n #Cov...        Positive
7   Was finally able to book a vaccination slot at...        Positive
8   18-44 #URBAN #Bengaluru #CovidVaccine Availabi...        Positive
9   Moderna, protein-based Covid jabs safe for use...        Positive
10  #Moderna is gearing up to halve the dose of it...         Neutral
11  Single #Dose of #SputnikV #Vaccine Triggers St...        Positive
12  All the more 💉 for the Brits. if lily-livered ...        Negative
13  @Ocugen Just put an news that effacy data out ...        Negative
14  ⚠️ #Moderna All death reports of less than 11y...        Negative
15  First shot #Moderna mild sore arm.\nSecond sho...        Negative
16  So, let’s all refer to it as the @NHS vaccine ...        Positive
17  It never fails: Every time there's a story abo...        Positive
18  Just got a call from my dead grandma. She died...        Negative
19  Just got my #Moderna #Booster at @Walgreens.  ...        Negative

In [10]:
data['text'].head(40)

0     @IsabelO20183051 The only consolation is that ...
1     My wife and I both got our 2nd #Moderna shots ...
2     Following the author of the study Tom Bayes, #...
3     560011, \nAGE 45+\n41-D1, 276-D2 slots,\n#COVA...
4               Happy 2nd Vaccine Day to me!!! #Moderna
5     @sepehrifar @hrw @MichaelARPage @ksadjadpour @...
6     Excellent work by Ayudh Volunteers! ...\n #Cov...
7     Was finally able to book a vaccination slot at...
8     18-44 #URBAN #Bengaluru #CovidVaccine Availabi...
9     Moderna, protein-based Covid jabs safe for use...
10    #Moderna is gearing up to halve the dose of it...
11    Single #Dose of #SputnikV #Vaccine Triggers St...
12    All the more 💉 for the Brits. if lily-livered ...
13    @Ocugen Just put an news that effacy data out ...
14    ⚠️ #Moderna All death reports of less than 11y...
15    First shot #Moderna mild sore arm.\nSecond sho...
16    So, let’s all refer to it as the @NHS vaccine ...
17    It never fails: Every time there's a story

In [11]:
!pip install emoji


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [12]:
!pip install contractions


In [13]:
# Nettoyage de texte simple
import re
import emoji
import contractions

def clean_text(text):
    text = text.lower()  # Mettre en minuscules
    text = emoji.demojize(text)  # Convertir les emojis en texte
    # Supprimer les emojis
    text = emoji.replace_emoji(text, replace='')   
    text = contractions.fix(text)  # Expander les contractions
    text = re.sub(r'@\w+', '', text)  # Supprimer les mentions Twitter
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text)  # Supprimer les URLs
    text = re.sub(r'#(\w+)', r'\1', text)  # Supprimer le symbole # et garder le mot après
    text = re.sub(r'[^\w\s\.,!?\'\"-]', '', text)  # Supprimer les caractères spéciaux, sauf ponctuation
    text = re.sub(r'\d+', '', text)  # Supprimer les chiffres
    text = re.sub(r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b', '', text)  # Supprimer les dates
    text = re.sub(r'[-/]', '', text)  # Supprimer les tirets et barres restantes
    text = re.sub(r'[^\w\s\.,!?\'\"-]', '', text) # Supprimer tous les caractères non alphanumériques
    text = re.sub(r'\s+', ' ', text).strip()  # Supprimer les espaces superflus

    return text



def clean_text(text):
    text = text.lower()  # Mettre en minuscules
    text = emoji.demojize(text)  # Convertir les emojis en texte
    text = contractions.fix(text)  # Expanser les contractions
    text = re.sub(r'@\w+', '', text)  # Supprimer les mentions Twitter
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text)  # Supprimer les URLs
    text = re.sub(r'#(\w+)', r'\1', text)  # Supprimer le symbole # et garder le mot après
    text = re.sub(r'[^\w\s\.,!?\'\"-]', '', text)
   
    # Supprimer les chiffres
    text = re.sub(r'\d+', '', text)  # Supprime tous les chiffres
    # Supprimer les dates (format : dd-mm-yyyy ou dd/mm/yyyy)
    text = re.sub(r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b', '', text)  # Supprime les dates

    # Supprimer les tirets ou barres qui auraient pu rester après la suppression des dates
    text = re.sub(r'[-/]', '', text)  # Supprime les tirets ou barres restantes
    # Supprimer tous les caractères restants qui ne sont pas des lettres ou des espaces
    text = re.sub(r'[^\w\s]', '', text)  # Supprime tous les caractères non alphanumériques
    # Supprimer les mots de longueur 1
    #text = re.sub(r'\b\w{1}\b', '', text)  # Supprime les mots uniques

    text = re.sub(r'\s+', ' ', text).strip()  # Supprimer les espaces superflus
    return text


In [14]:
# Exemple d'utilisation
texte_exemple = """
I'm very 1233 12-12-2002 happy  with the service! You,, can't *** :)  visit our @gmil site http://example.com
#satisfaction @service 😊👍
"""

texte_nettoye = clean_text(texte_exemple)
print(texte_nettoye)

i am very happy with the service! you,, cannot visit our site satisfaction smiling_face_with_smiling_eyesthumbs_up


In [15]:
# Nettoyer le texte
data['cleaned_text_bert'] = data['text'].apply(clean_text)

In [16]:
data

text Sentiment_vader  \
0      @IsabelO20183051 The only consolation is that ...        Positive   
1      My wife and I both got our 2nd #Moderna shots ...        Positive   
2      Following the author of the study Tom Bayes, #...        Negative   
3      560011, \nAGE 45+\n41-D1, 276-D2 slots,\n#COVA...         Neutral   
4                Happy 2nd Vaccine Day to me!!! #Moderna        Positive   
...                                                  ...             ...   
86810  #ocgn is the stock 😎 , huge potential coming v...        Positive   
86811  Some have noted Modi’s visit goes against curr...         Neutral   
86812  Pincode: 560027\nCOVAXIN - Dose 2: 50 slots\n\...         Neutral   
86813  [18:03:42] 400602, 16-07-2021, 61 dose2 at Hig...         Neutral   
86814  Pincode: 560003\nCOVAXIN - Dose 2: 20 slots\n\...         Neutral   

                                       cleaned_text_bert  
0      the only consolation is that good number of sp...  
1      my wife and i both got our nd moderna shots ye...  
2      following the author of the study tom bayes, c...  
3      , age d, d slots, covaxin on apollo jayanagarc...  
4                  happy nd vaccine day to me!!! moderna  
...                                                  ...  
86810  ocgn is the stock smiling_face_with_sunglasses...  
86811  some have noted modis visit goes against curre...  
86812  pincode covaxin dose slots age and above all a...  
86813  , , dose at highway hospital tmc thane, covaxi...  
86814  pincode covaxin dose slots age and above all a...  

[86815 rows x 3 columns]

In [17]:
# Exemple de mappage pour convertir les sentiments en valeurs numériques
sentiment_mapping = {
    'Positive': 2,
    'Neutral': 1,
    'Negative': 0
}

# Appliquer le mappage à la colonne des sentiments
data['sentiment_label'] = data['Sentiment_vader'].map(sentiment_mapping)

In [18]:
data.head()

text Sentiment_vader  \
0  @IsabelO20183051 The only consolation is that ...        Positive   
1  My wife and I both got our 2nd #Moderna shots ...        Positive   
2  Following the author of the study Tom Bayes, #...        Negative   
3  560011, \nAGE 45+\n41-D1, 276-D2 slots,\n#COVA...         Neutral   
4            Happy 2nd Vaccine Day to me!!! #Moderna        Positive   

                                   cleaned_text_bert  sentiment_label  
0  the only consolation is that good number of sp...                2  
1  my wife and i both got our nd moderna shots ye...                2  
2  following the author of the study tom bayes, c...                0  
3  , age d, d slots, covaxin on apollo jayanagarc...                1  
4              happy nd vaccine day to me!!! moderna                2

In [19]:

# Afficher la répartition des sentiments après suréchantillonnage
print(data['sentiment_label'].value_counts())


sentiment_label
0    29513
2    28651
1    28651
Name: count, dtype: int64


In [20]:
data.head(20)

text Sentiment_vader  \
0   @IsabelO20183051 The only consolation is that ...        Positive   
1   My wife and I both got our 2nd #Moderna shots ...        Positive   
2   Following the author of the study Tom Bayes, #...        Negative   
3   560011, \nAGE 45+\n41-D1, 276-D2 slots,\n#COVA...         Neutral   
4             Happy 2nd Vaccine Day to me!!! #Moderna        Positive   
5   @sepehrifar @hrw @MichaelARPage @ksadjadpour @...        Negative   
6   Excellent work by Ayudh Volunteers! ...\n #Cov...        Positive   
7   Was finally able to book a vaccination slot at...        Positive   
8   18-44 #URBAN #Bengaluru #CovidVaccine Availabi...        Positive   
9   Moderna, protein-based Covid jabs safe for use...        Positive   
10  #Moderna is gearing up to halve the dose of it...         Neutral   
11  Single #Dose of #SputnikV #Vaccine Triggers St...        Positive   
12  All the more 💉 for the Brits. if lily-livered ...        Negative   
13  @Ocugen Just put an news that effacy data out ...        Negative   
14  ⚠️ #Moderna All death reports of less than 11y...        Negative   
15  First shot #Moderna mild sore arm.\nSecond sho...        Negative   
16  So, let’s all refer to it as the @NHS vaccine ...        Positive   
17  It never fails: Every time there's a story abo...        Positive   
18  Just got a call from my dead grandma. She died...        Negative   
19  Just got my #Moderna #Booster at @Walgreens.  ...        Negative   

                                    cleaned_text_bert  sentiment_label  
0   the only consolation is that good number of sp...                2  
1   my wife and i both got our nd moderna shots ye...                2  
2   following the author of the study tom bayes, c...                0  
3   , age d, d slots, covaxin on apollo jayanagarc...                1  
4               happy nd vaccine day to me!!! moderna                2  
5   khamenei "us vaccines are likely unsafe and sh...                0  
6   excellent work by ayudh volunteers! ... covidh...                2  
7   was finally able to book a vaccination slot at...                2  
8   urban bengaluru covidvaccine availability for ...                2  
9   moderna, proteinbased covid jabs safe for use ...                2  
10  moderna is gearing up to halve the dose of its...                1  
11  single dose of sputnikv vaccine triggers stron...                2  
12  all the more syringe for the brits. if lilyliv...                0  
13  just put an news that effacy data out soon!!! ...                0  
14  warning moderna all death reports of less than...                0  
15  first shot moderna mild sore arm. second shot,...                0  
16  so, let us all refer to it as the vaccine roll...                2  
17  it never fails every time there is a story abo...                2  
18  just got a call from my dead grandma. she died...                0  
19  just got my moderna booster at . no line, very...                0

In [21]:
# Afficher les lignes où la colonne 'text' est vide ou contient seulement des espaces
empty_text_rows = data[data['cleaned_text_bert'].str.strip() == '']
print(empty_text_rows)


                                                    text Sentiment_vader  \
474    @MoHFW_INDIA @PMOIndia @mansukhmandviya @ianur...         Neutral   
551    @jsr221016 @KellieADennis74 @Zippy18 @SCTJ9 @S...         Neutral   
640    @airnews_ghy @diprassam @UGBrahma @manassarma ...         Neutral   
675    @mybmc @PMOIndia @MoHFW_INDIA @CMOMaharashtra ...         Neutral   
757    @SuchitraElla @BharatBiotech @icmr_niv @MoHFW_...         Neutral   
...                                                  ...             ...   
85518  @MarciaBunney @Goddess98465465 @BernadetteNTF ...         Neutral   
85588  @NutLoOOse @RanjitWickreme2 @ChinaEmbSL @Naren...         Neutral   
85634  @JanetBrown980 @izzyisdancing @jodyvance @keit...         Neutral   
85778  @DrBirinderSingh @NeetuGarcha @anitabathe @Son...         Neutral   
85946  @IrinaIrusja @cabinetofficeuk @Simonhartmp @Bo...         Neutral   

      cleaned_text_bert  sentiment_label  
474                                    1  
5

In [22]:
# Supprimer les lignes où la colonne 'text' est nulle ou vide
data = data[~data['cleaned_text_bert'].isnull() & (data['cleaned_text_bert'].str.strip() != '')]


In [23]:
print(data['cleaned_text_bert'].str.len().describe())

count    86496.000000
mean        91.840906
std         25.826496
min          1.000000
25%         75.000000
50%         96.000000
75%        110.000000
max       1076.000000
Name: cleaned_text_bert, dtype: float64


In [24]:
# Vérifiez s'il y a des valeurs manquantes après le mappage
if data['sentiment_label'].isnull().any():
    print("Attention: Certaines valeurs de la colonne Sentiment n'ont pas été mappées.")
    print(data[data['sentiment_label'].isnull()]['Sentiment'].unique())

In [25]:
data['cleaned_text_bert']

0        the only consolation is that good number of sp...
1        my wife and i both got our nd moderna shots ye...
2        following the author of the study tom bayes, c...
3        , age d, d slots, covaxin on apollo jayanagarc...
4                    happy nd vaccine day to me!!! moderna
                               ...                        
86810    ocgn is the stock smiling_face_with_sunglasses...
86811    some have noted modis visit goes against curre...
86812    pincode covaxin dose slots age and above all a...
86813    , , dose at highway hospital tmc thane, covaxi...
86814    pincode covaxin dose slots age and above all a...
Name: cleaned_text_bert, Length: 86496, dtype: object

In [26]:
# Initialiser le tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [27]:
# Trouver l'indice du tweet avec le plus grand nombre de caractères
max_length_index = data['cleaned_text_bert'].str.len().idxmax()

# Afficher le tweet avec le plus grand nombre de caractères
longest_tweet = data['cleaned_text_bert'].iloc[max_length_index]
length_of_longest_tweet = len(longest_tweet)

print(f"Le tweet le plus long a {length_of_longest_tweet} caractères :\n{longest_tweet}")


Le tweet le plus long a 142 caractères :
got jabbed..... loading antibodies.... flexed_bicepshigh_voltagehigh_voltage covaxin best day to get to do it on happydoctorsday and yeah happ


In [28]:
# Tokenization des données
encodings = tokenizer.batch_encode_plus(
    data['cleaned_text_bert'].tolist(),  # Liste des textes à tokeniser
    truncation=True,                      # Tronquer les textes qui dépassent max_length
    padding='longest',                         # Ajouter du padding pour que tous les textes aient la même longueur
    max_length=128,                       # Longueur maximale des séquences de tokens
    return_tensors='pt'                  # Retourner des tenseurs au format PyTorch
)

input_ids = encodings['input_ids']      # Récupérer les identifiants de tokens
attention_masks = encodings['attention_mask']  # Récupérer les masques d'attention
labels = torch.tensor(data['sentiment_label'].values)  # Récupérer les étiquettes des sentiments


In [29]:
labels

tensor([2, 2, 0,  ..., 1, 1, 1])

In [30]:
# Créer un DataLoader pour l'entraînement
dataset = TensorDataset(input_ids, attention_masks, labels)

# Diviser le dataset en ensemble d'entraînement et de validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# Créer les DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)


In [31]:
# Charger le modèle BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [32]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# Définir les hyperparamètres de l'entraînement
epochs = 10
patience = 3  # Nombre d'époques à attendre pour l'amélioration
best_val_loss = float('inf')
patience_counter = 0

# Initialiser l'optimiseur AdamW
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

for epoch in range(epochs):
    model.train()
    total_train_loss = 0.0
    correct_train_predictions = 0
    total_train_predictions = 0

    # Entraînement
    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Calculer les prédictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        correct_train_predictions += (predictions == labels).sum().item()
        total_train_predictions += labels.size(0)

    avg_train_loss = total_train_loss / len(train_dataloader)
    train_accuracy = correct_train_predictions / total_train_predictions
    print(f"Époque {epoch + 1}, Perte moyenne d'entraînement: {avg_train_loss:.6f}, Précision d'entraînement: {train_accuracy:.8f}")

    # Validation
    model.eval()
    total_val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

            # Calculer les prédictions
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct_val_predictions += (predictions == labels).sum().item()
            total_val_predictions += labels.size(0)

            # Stocker les prédictions et les labels pour les métriques
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = correct_val_predictions / total_val_predictions
    print(f"Époque {epoch + 1}, Perte moyenne de validation: {avg_val_loss:.6f}, Précision de validation: {val_accuracy:.8f}")

    # Calculer et afficher les métriques
    print("Classification Report:")
    print(classification_report(all_labels, all_predictions, target_names=['Negative', 'Neutral', 'Positive']))

    print("Confusion Matrix:")
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    print(conf_matrix)

    # Vérifier pour l'arrêt précoce
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model.pt')
        torch.save(model.state_dict(), '/kaggle/working/best_model.pt')
        print("Meilleur modèle enregistré")

        # Sauvegarder le tokenizer
        tokenizer.save_pretrained('tokenizer_directory')
        tokenizer.save_pretrained('/kaggle/working/tokenizer_directory')
    else:
        patience_counter += 1
        print(f"Aucune amélioration depuis {patience_counter} époques.")

        # Si le compteur de patience atteint le seuil, arrêter l'entraînement
        if patience_counter >= patience:
            print("Arrêt précoce déclenché.")
            break

print("Entraînement terminé.")


Époque 1, Perte moyenne d'entraînement: 0.223979, Précision d'entraînement: 0.91966299
Époque 1, Perte moyenne de validation: 0.109917, Précision de validation: 0.96473988
Classification Report:
              precision    recall  f1-score   support

    Negative       0.95      0.96      0.95      5799
     Neutral       0.98      0.97      0.97      5690
    Positive       0.96      0.97      0.97      5811

    accuracy                           0.96     17300
   macro avg       0.96      0.96      0.96     17300
weighted avg       0.96      0.96      0.96     17300

Confusion Matrix:
[[5562   86  151]
 [ 143 5492   55]
 [ 155   20 5636]]
Meilleur modèle enregistré
Époque 2, Perte moyenne d'entraînement: 0.080543, Précision d'entraînement: 0.97492630
Époque 2, Perte moyenne de validation: 0.091722, Précision de validation: 0.97294798
Classification Report:
              precision    recall  f1-score   support

    Negative       0.98      0.95      0.96      5799
     Neutral       0

In [45]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix



# Charger le modèle sauvegardé
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.load_state_dict(torch.load('best_model.pt'))  # Charger les poids sauvegardés
model.eval()  # Mettre le modèle en mode évaluation

# Charger le tokenizer sauvegardé
tokenizer = BertTokenizer.from_pretrained('tokenizer_directory')
# Envoyer le modèle sur le GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assurez-vous que le modèle est en mode évaluation
model.eval()

import pandas as pd
test_data=pd.read_csv('/kaggle/input/test-data2/test_dataset3.csv')





# Charger l'ensemble de test


test_data['cleaned_text_bert'] = test_data['text'].apply(clean_text)

test_texts = test_data['cleaned_text_bert'].tolist() 


# Exemple de mappage pour convertir les sentiments en valeurs numériques
sentiment_mapping = {
    'Positive': 2,
    'Neutral': 1,
    'Negative': 0
}

test_labels = test_data['Sentiment_vader'].map(sentiment_mapping)  # Les véritables étiquettes des données de test
test_labels=test_labels.tolist()
# Tokenization des données de test
encodings = tokenizer.batch_encode_plus(
    test_texts,
    truncation=True,
    padding='longest',  # Ajouter du padding pour que tous les textes aient la même longueur
    max_length=128,  # Utiliser la même longueur que pendant l'entraînement
    return_tensors='pt'
)



# Convertir les données en TensorDataset et DataLoader
input_ids = encodings['input_ids']
attention_masks = encodings['attention_mask']
labels = torch.tensor(test_labels)

test_dataset = TensorDataset(input_ids, attention_masks, labels)
test_loader = DataLoader(test_dataset, batch_size=32)  # Batch size peut être ajusté

# Evaluation du modèle sur l'ensemble de test
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_masks, labels = batch

        # Envoyer les données au GPU si disponible
        input_ids = input_ids.to('cuda') if torch.cuda.is_available() else input_ids
        attention_masks = attention_masks.to('cuda') if torch.cuda.is_available() else attention_masks
        labels = labels.to('cuda') if torch.cuda.is_available() else labels

        # Prédictions
        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        all_preds.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Afficher les résultats : rapport de classification et matrice de confusion
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=['Negative', 'Neutral', 'Positive']))

print("Confusion Matrix:")
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Classification Report:
              precision    recall  f1-score   support

    Negative       0.95      0.98      0.96      7378
     Neutral       0.99      0.97      0.98      7163
    Positive       0.98      0.97      0.97      7163

    accuracy                           0.97     21704
   macro avg       0.97      0.97      0.97     21704
weighted avg       0.97      0.97      0.97     21704

Confusion Matrix:
[[7207   67  104]
 [ 180 6954   29]
 [ 217   28 6918]]


In [47]:
import shutil
from IPython.display import FileLink

# Compresser le dossier en fichier .zip
shutil.make_archive('tokenizer_directory', 'zip', '/kaggle/working/tokenizer_directory')

# Créer un lien pour télécharger le fichier .zip
display(FileLink('tokenizer_directory.zip'))
display(FileLink('best_model.pt'))
display(FileLink('state.db'))

/kaggle/working/tokenizer_directory.zip

/kaggle/working/best_model.pt

/kaggle/working/state.db